In [1]:
from sims_pars import bayes_net_from_script, sample, evaluate_nodes
from sims_pars.bayesnet import Chromosome

In [2]:
script = '''
PCore Test {
    k = 5
    a ~ unif(1, k)
    b ~ norm(0, 1)
    c ~ norm(b, a)
}
'''

In [3]:
bn = bayes_net_from_script(script)

In [4]:
bn.RVRoots

['b', 'a']

In [12]:
def sample_prior(bn, exo=None):
    p = sample(bn, cond=exo)
    lp = evaluate_nodes(bn, p)
    ch = Chromosome(p, lp)
    return ch
        
sample_prior(bn)

k: 5, b: -0.495143, a: 1.07374, c: -0.278576

In [1]:
chr

<function chr(i, /)>

In [13]:
%timeit sample_prior(bn)

4.58 ms ± 306 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
exo = {'k': 10}

p = sample(bn, cond=exo)
chr = Chromosome(p, evaluate_nodes(bn, p))
chr.to_json()

{'Locus': {'k': 10,
  'b': -0.2772143823286646,
  'a': 8.240536001100262,
  'c': -1.7866641423876632},
 'LogProb': -6.199367249762359}

In [7]:
Chromosome.from_json(chr.to_json())

k: 5, b: -2.02207, a: 1.27073, c: -4.09875

In [11]:
chr.impulse({'a': 2.0}, bn)
chr

k: 10, b: -0.277214, a: 2, c: -0.687725

In [2]:
class Test(AbsObjectiveSimBased):
    def __init__(self):
        bn = bayes_net_from_script('''
        PCore Normal2 {
            r_die_tb_untx ~ unif(0.07, 0.19)
            rr_die_asym ~ unif(0, 1)

            prev_rural ~ unif(0.001, 0.01)
            prev_urban ~ unif(0.001, 0.01)
            prev_slum ~ unif(0.001, 0.01)

            // Life history
            r_sc ~ unif(0.15, 0.25)  # self-cure

            // Care-seeking and diagnosis
            rr_det_urban ~ unif(0.5, 5)
            rr_det_slum ~ unif(0.5, 5)

            p_dx_pri_frac ~ unif(0.5, 1)
            p_dx_pub = 0.83
            p_dx_eng = (p_dx_pub + p_dx_pri) / 2
            p_dx_pri = p_dx_pub * p_dx_pri_frac

            p_txi_pub = 0.88
            p_txi_eng = 0.88
            p_txi_pri = 0.75

            p_public ~ unif(0.3, 0.8)
            p_eng_rural_frac ~ unif(0, 1)
            p_eng_rural = p_eng_urban * p_eng_rural_frac
            p_eng_urban ~ unif(0, 1)
            p_eng_slum = p_eng_urban
            r_acf ~ unif(0, 0.05)

            ppv = 0.85

            // Treatment
            r_tx_succ = 2
            dur_succ_ds = 0.5
            dur_succ_dr = 2
            x ~ binom(size=60, prob = 0.5)
        }
        ''')
        AbsObjectiveSimBased.__init__(self, bn)

    def simulate(self, pars):
        return 'test'

    def link_likelihood(self, sim):
        return 1

In [6]:
p = obj.sample_prior()
obj.evaluate(p)
p.to_json()

{'Locus': {'r_die_tb_untx': 0.15975179206386825,
  'rr_die_asym': 0.9409856140974366,
  'prev_rural': 0.00815563702514403,
  'prev_urban': 0.009387385314785703,
  'prev_slum': 0.007118297913251721,
  'r_sc': 0.1686733609685655,
  'rr_det_urban': 2.0127140244006423,
  'rr_det_slum': 2.546199829864127,
  'p_dx_pri_frac': 0.6377158017219158,
  'p_dx_pub': 0.83,
  'p_txi_pub': 0.88,
  'p_txi_eng': 0.88,
  'p_txi_pri': 0.75,
  'p_public': 0.688785903480731,
  'p_eng_rural_frac': 0.77026796852874,
  'p_eng_urban': 0.9850912367598427,
  'r_acf': 0.041830390191166605,
  'ppv': 0.85,
  'r_tx_succ': 2,
  'dur_succ_ds': 0.5,
  'dur_succ_dr': 2,
  'x': 27,
  'p_dx_pri': 0.5293041154291901,
  'p_eng_rural': 0.7587842257544681,
  'p_eng_slum': 0.9850912367598427,
  'p_dx_eng': 0.679652057714595},
 'LogPrior': 17.355649520740293,
 'LogLikelihood': 1}

In [7]:
obj.FreeParameters

['r_die_tb_untx',
 'rr_die_asym',
 'prev_rural',
 'prev_urban',
 'prev_slum',
 'r_sc',
 'rr_det_urban',
 'rr_det_slum',
 'p_dx_pri_frac',
 'p_public',
 'p_eng_rural_frac',
 'p_eng_urban',
 'r_acf']

In [8]:
step_sizes = {
    'prev_rural': 0.001,
    'prev_urban': 0.001,
    'prev_slum': 0.001,
    'r_sc': 0.05,
    'rr_det_urban': 0.1,
    'rr_det_slum': 0.1,
    'p_dx_pri_frac': 0.1,
    'p_public': 0.1,
    'p_eng_rural_frac': 0.5
}

def mutate1(p0, sizes):
    p = p0.clone()
    changes = {k: v + sizes[k] for k, v in p.Locus.items() if k in sizes}

    p.impulse(changes)
    obj.evaluate_prior(p)
    return p

%timeit mutate1(p, step_sizes)

8.4 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
def mutate2(p0, sizes):
    changes = {k: v + sizes[k] for k, v in p0.Locus.items() if k in sizes}

    return obj.serve(changes)

%timeit mutate2(p, step_sizes)

10.9 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
obj.Domain

[Domain(Name='r_die_tb_untx', Type='Double', Lower=0.07, Upper=0.19),
 Domain(Name='rr_die_asym', Type='Double', Lower=0.0, Upper=1.0),
 Domain(Name='prev_rural', Type='Double', Lower=0.001, Upper=0.010000000000000002),
 Domain(Name='prev_urban', Type='Double', Lower=0.001, Upper=0.010000000000000002),
 Domain(Name='prev_slum', Type='Double', Lower=0.001, Upper=0.010000000000000002),
 Domain(Name='r_sc', Type='Double', Lower=0.15, Upper=0.25),
 Domain(Name='rr_det_urban', Type='Double', Lower=0.5, Upper=5.0),
 Domain(Name='rr_det_slum', Type='Double', Lower=0.5, Upper=5.0),
 Domain(Name='p_dx_pri_frac', Type='Double', Lower=0.5, Upper=1.0),
 Domain(Name='p_public', Type='Double', Lower=0.3, Upper=0.8),
 Domain(Name='p_eng_rural_frac', Type='Double', Lower=0.0, Upper=1.0),
 Domain(Name='p_eng_urban', Type='Double', Lower=0.0, Upper=1.0),
 Domain(Name='r_acf', Type='Double', Lower=0.0, Upper=0.05)]